In [1]:
#to use API youtube
from googleapiclient.discovery import build

#to transform Data
import pandas as pd

#To manipulate dates
import isodate

In [2]:
api_key = 'Put your API KEY from google here'

In [3]:
channel_ids = ['UCyIeTK4ouP6v3dlwXK-b3cg' #Cerrado Aquarios
               ]

In [4]:
api_service_name = "youtube"
api_version = "v3"


    # Get credentials and create an API client
youtube = build(
        api_service_name, api_version, developerKey=api_key)

In [5]:
# Get channel statistics that I seted and create a Dataframe
def get_channel_stats(youtube, channel_ids):

    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()

    # loop through items
    for item in response['items']:
     data = {'channelName': item['snippet'] ['title'],
            'subscribers': item['statistics'] ['subscriberCount'],
            'views': item['statistics'] ['viewCount'],
             'totalVideos': item['statistics'] ['videoCount'],
            'playlistId': item['contentDetails'] ['relatedPlaylists'] ['uploads']
            }

    all_data.append(data)

    return pd.DataFrame(all_data)

In [6]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [7]:
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Cerrado Aquários,2020,270424,56,UUyIeTK4ouP6v3dlwXK-b3cg


In [8]:
#you can find de Playlist_ID here

request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        playlistId="UUyIeTK4ouP6v3dlwXK-b3cg"
    )
response = request.execute()

print(response)

{'kind': 'youtube#playlistItemListResponse', 'etag': 'JgGdGP4OP4WiQKWjJ-y990oH-wQ', 'nextPageToken': 'EAAaBlBUOkNBVQ', 'items': [{'kind': 'youtube#playlistItem', 'etag': 'k5VqIHwHWVPm36CgmyZGF0mlw60', 'id': 'VVV5SWVUSzRvdVA2djNkbHdYSy1iM2NnLjBBcDl2SzI0MURJ', 'snippet': {'publishedAt': '2022-02-26T16:01:02Z', 'channelId': 'UCyIeTK4ouP6v3dlwXK-b3cg', 'title': 'Transmissão ao vivo de Cerrado Aquários', 'description': '', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/0Ap9vK241DI/default_live.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/0Ap9vK241DI/mqdefault_live.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/0Ap9vK241DI/hqdefault_live.jpg', 'width': 480, 'height': 360}, 'standard': {'url': 'https://i.ytimg.com/vi/0Ap9vK241DI/sddefault_live.jpg', 'width': 640, 'height': 480}, 'maxres': {'url': 'https://i.ytimg.com/vi/0Ap9vK241DI/maxresdefault_live.jpg', 'width': 1280, 'height': 720}}, 'channelTitle': 'Cerrado Aquários'

In [9]:
#I defined here max results = 50 ids, but you change like you wish 
playlist_id = 'PLZQU7BP-EQuolApj2RfrPqePV64QHas4l'

def get_video_ids(youtube, playlist_id):

    video_ids = []

    request = youtube.playlistItems().list(
        part='snippet, contentDetails',
        playlistId=playlist_id,
        maxResults = 50 
    )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails'] ['videoId'])

    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()    

    return video_ids   


In [10]:
video_ids = get_video_ids(youtube, playlist_id)

In [11]:
#The list of Ids from playlist

video_ids

['DhiFcUMw39A', 'K1gbrySZgM4', 'EW0KoRkxDlg', '1wBheqYbhAc']

In [12]:
#Get video statistics of all videos with given IDs (Here you can defined the columns that you want to use )
def get_video_details(youtube, video_ids):

    all_video_info = []
    
    request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=video_ids
        )
    response = request.execute() 

    for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)      


In [13]:
video_df = get_video_details(youtube, video_ids)

In [14]:
video_df

,video_id,channelTitle,title,description,publishedAt,viewCount,likeCount,commentCount,duration,definition
0,DhiFcUMw39A,Cerrado Aquários,Betta de Linhagem - Plakat Pink Salamander - ...,Para quem deseja adquirir esse Betta ou outros...,2017-10-26T23:05:36Z,6802,292,31,PT3M9S,hd
1,K1gbrySZgM4,Cerrado Aquários,Meu Peixe Betta Dragon,Siga nosso instagram: https://www.instagram.co...,2017-05-23T22:00:58Z,3085,80,18,PT59S,hd
2,EW0KoRkxDlg,Cerrado Aquários,Peixe Betta - Aquário Ideal,Siga nosso instagram: https://www.instagram.co...,2017-02-05T14:30:35Z,54907,1616,328,PT7M48S,hd
3,1wBheqYbhAc,Cerrado Aquários,"Montagem de Aquário para peixe Betta Básico, b...",Siga nosso instagram: https://www.instagram.co...,2017-10-14T22:35:23Z,17930,503,89,PT12M29S,hd


In [15]:
#Verifying the nulls
video_df.isnull().any()

video_id        False
channelTitle    False
title           False
description     False
publishedAt     False
viewCount       False
likeCount       False
commentCount    False
duration        False
definition      False
dtype: bool

In [16]:
#Verifying the types
video_df.dtypes

video_id        object
channelTitle    object
title           object
description     object
publishedAt     object
viewCount       object
likeCount       object
commentCount    object
duration        object
definition      object
dtype: object

In [17]:
#Convert columns to numeric
numeric_cols = ['viewCount','likeCount', 'commentCount']
video_df[numeric_cols] = video_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [18]:
#take off the time and let only the date
video_df['publishedAt'] = pd.to_datetime(video_df['publishedAt'])
video_df['publishedAt'] = video_df['publishedAt'].dt.date

In [19]:
#Verifying the types(update)
video_df.dtypes

video_id        object
channelTitle    object
title           object
description     object
publishedAt     object
viewCount        int64
likeCount        int64
commentCount     int64
duration        object
definition      object
dtype: object

In [20]:
# convert duration to Seconds
video_df['durationSec'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x))
video_df['durationSec'] = video_df['durationSec'].astype('timedelta64[s]')

In [22]:
#Remove duration column
video_df = video_df.drop('duration', axis=1)
video_df

,video_id,channelTitle,title,description,publishedAt,viewCount,likeCount,commentCount,definition,durationSec
0,DhiFcUMw39A,Cerrado Aquários,Betta de Linhagem - Plakat Pink Salamander - ...,Para quem deseja adquirir esse Betta ou outros...,2017-10-26,6802,292,31,hd,189.0
1,K1gbrySZgM4,Cerrado Aquários,Meu Peixe Betta Dragon,Siga nosso instagram: https://www.instagram.co...,2017-05-23,3085,80,18,hd,59.0
2,EW0KoRkxDlg,Cerrado Aquários,Peixe Betta - Aquário Ideal,Siga nosso instagram: https://www.instagram.co...,2017-02-05,54907,1616,328,hd,468.0
3,1wBheqYbhAc,Cerrado Aquários,"Montagem de Aquário para peixe Betta Básico, b...",Siga nosso instagram: https://www.instagram.co...,2017-10-14,17930,503,89,hd,749.0
